In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from skimage.metrics import structural_similarity as ssim


In [3]:
%cd Vision/data/Outdoor/color_correct/


c:\Users\hasan\OneDrive\Documents\GitHub\Vision-Transformer-Image-Dehazing-hasanhd555\Vision\data\Outdoor\color_correct


'c:\\Users\\hasan\\OneDrive\\Documents\\GitHub\\Vision-Transformer-Image-Dehazing-hasanhd555\\Vision\\data\\Outdoor\\color_correct'

In [8]:
%pwd

'c:\\Users\\hasan\\OneDrive\\Documents\\GitHub\\Vision-Transformer-Image-Dehazing-hasanhd555\\Vision\\data\\Outdoor\\color_correct'

In [23]:

# Define a custom dataset class to load noisy and ground truth images

import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
class ColorCorrectionDataset(Dataset):
    def __init__(self, noisy_dir, gt_dir, transform=None):
        self.noisy_dir = noisy_dir
        self.gt_dir = gt_dir
        self.transform = transform
        
        # Get the list of filenames in both directories
        self.noisy_images = os.listdir(noisy_dir)
        self.gt_images = os.listdir(gt_dir)
        
        assert len(self.noisy_images) == len(self.gt_images), "Number of noisy and GT images must be the same."

    def __getitem__(self, index):
        noisy_img_path = os.path.join(self.noisy_dir, self.noisy_images[index])
        gt_img_path = os.path.join(self.gt_dir, self.gt_images[index])
        
        # Open images
        noisy_img = Image.open(noisy_img_path).convert('RGB')
        gt_img = Image.open(gt_img_path).convert('RGB')
        
        # Apply transforms if specified
        if self.transform:
            noisy_img = self.transform(noisy_img)
            gt_img = self.transform(gt_img)
            
        return noisy_img, gt_img

    def __len__(self):
        return len(self.noisy_images)


In [38]:

# Define the convolutional neural network architecture
import torch
import torch.nn as nn
import torch.nn.functional as F

class ColorCorrectionCNN(nn.Module):
    def __init__(self):
        super(ColorCorrectionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(64)
        self.conv9 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = self.relu(self.bn5(self.conv5(x)))
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.relu(self.bn6(self.conv6(x)))
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.relu(self.bn7(self.conv7(x)))
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.relu(self.bn8(self.conv8(x)))
        x = self.conv9(x)
        return x


# Define SSIM loss function
class SSIMLoss(nn.Module):
    def __init__(self, win_size=11):
        super(SSIMLoss, self).__init__()
        self.win_size = win_size

    def forward(self, output, target):
        output = output.permute(0, 2, 3, 1).cpu().detach().numpy()
        target = target.permute(0, 2, 3, 1).cpu().detach().numpy()
        ssim_loss = 1 - ssim(output, target, win_size=self.win_size, multichannel=True,channel_axis=3,data_range=1)
        return torch.tensor(ssim_loss, requires_grad=True)


In [39]:

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create datasets and dataloaders
train_dataset = ColorCorrectionDataset(noisy_dir='train/noisy', gt_dir='train/GT', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = ColorCorrectionDataset(noisy_dir='test/noisy', gt_dir='test/GT', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

#load one image
noisy_img, gt_img = train_dataset[0]
print(noisy_img.shape, gt_img.shape)

torch.Size([3, 256, 256]) torch.Size([3, 256, 256])


In [41]:

# Create the model, loss function, and optimizer
model = ColorCorrectionCNN().to(device)
criterion = SSIMLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

loss_values = []
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (noisy_img, gt_img) in enumerate(train_loader):
        noisy_img, gt_img = noisy_img.to(device), gt_img.to(device)
        
        optimizer.zero_grad()
        outputs = model(noisy_img)
        loss = criterion(outputs, gt_img)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/10:.4f}")
            running_loss = 0.0
    loss_values.append(running_loss)

print("Training finished.")


Epoch [1/10], Step [10/10], Loss: 0.9956
Epoch [2/10], Step [10/10], Loss: 0.9951
Epoch [3/10], Step [10/10], Loss: 0.9957
Epoch [4/10], Step [10/10], Loss: 0.9956
Epoch [5/10], Step [10/10], Loss: 0.9951


In [ ]:

# Evaluate the model on the test set
model.eval()
total_ssim_loss = 0.0
with torch.no_grad():
    for i, (noisy_img, gt_img) in enumerate(test_loader):
        noisy_img, gt_img = noisy_img.to(device), gt_img.to(device)
        outputs = model(noisy_img)
        loss = criterion(outputs, gt_img)
        total_ssim_loss += loss.item()

print(f"Average SSIM loss on test set: {total_ssim_loss/len(test_loader):.4f}")
import matplotlib.pyplot as plt
import numpy as np


# Visualize the loss over epochs
plt.plot(np.arange(1, num_epochs+1), loss_values)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()





In [ ]:

# Save the trained model
torch.save(model.state_dict(), 'color_correction_model.pth')
print("Model saved.")
